## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [15]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline



# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [16]:
#train_data['Promo'] = train_data.apply(lambda x: max(len(x['a']), len(x['b'])), axis=1)
#train_data.loc[train_data['Promotion'] == 'Yes', 'Promo'] ==1
train_data['Promo'] = train_data['Promotion'].apply(lambda x: 1 if x == 'Yes' else 0)

In [17]:
train_data.head(25)

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7,Promo
0,1,No,0,2,30.443518,-1.165083,1,1,3,2,0
1,3,No,0,3,32.159350,-0.645617,2,3,2,2,0
2,4,No,0,2,30.431659,0.133583,1,1,4,2,0
3,5,No,0,0,26.588914,-0.212728,2,1,4,2,0
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2,1
5,9,No,0,1,41.189415,-0.905350,2,4,4,1,0
6,11,No,0,1,40.690409,1.085939,2,2,1,2,0
7,14,No,0,2,25.971529,-1.424817,1,3,3,2,0
8,15,Yes,0,2,31.930423,0.393317,2,3,1,2,1
9,16,No,0,1,27.474650,0.566472,2,2,3,2,0


In [18]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need
train_data.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7,Promo
count,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000
mean,62970.972413,0.012303,1.500662,29.973600,0.000190,1.679608,2.327643,2.502898,1.701694,0.501147
std,36418.440539,0.110234,0.868234,5.010626,1.000485,0.466630,0.841167,1.117349,0.457517,0.500002
min,1.000000,0.000000,0.000000,7.104007,-1.684550,1.000000,1.000000,1.000000,1.000000,0.000000
25%,31467.250000,0.000000,1.000000,26.591501,-0.905350,1.000000,2.000000,2.000000,1.000000,0.000000
50%,62827.500000,0.000000,2.000000,29.979744,-0.039572,2.000000,2.000000,3.000000,2.000000,1.000000
75%,94438.750000,0.000000,2.000000,33.344593,0.826206,2.000000,3.000000,4.000000,2.000000,1.000000
max,126184.000000,1.000000,3.000000,50.375913,1.691984,2.000000,4.000000,4.000000,2.000000,1.000000


In [13]:
train_data.groupby('Promotion').agg({'V1': ['min', 'max'],'V2': ['min', 'max'],'V3': ['min', 'max'],'V4': ['min', 'max'],'V5': ['min', 'max'],'V6': ['min', 'max'],'V7': ['min', 'max'], 'purchase': 'sum'})


V1            V2                  V3            V4      V5      V6  \
          min max       min        max      min       max min max min max min   
Promotion                                                                       
No          0   3  8.939305  50.375913 -1.68455  1.691984   1   2   1   4   1   
Yes         0   3  7.104007  49.823708 -1.68455  1.691984   1   2   1   4   1   

               V7     purchase  
          max min max      sum  
Promotion                       
No          4   1   2      319  
Yes         4   1   2      721

In [30]:
train_int_data=train_data[['purchase','Promo','V2','V3']]
#df[['color', 'food', 'score']]

In [31]:
train_data.groupby('purchase').mean()

,ID,V1,V2,V3,V4,V5,V6,V7,Promo
purchase,,,,,,,,,
0,62942.917383,1.501138,29.974547,0.000905,1.677929,2.326910,2.50309,1.701751,0.498754
1,65223.305769,1.462500,29.897554,-0.057137,1.814423,2.386538,2.48750,1.697115,0.693269


In [36]:
train_data.groupby('purchase').Promo.sum()

purchase
0    41643
1      721
Name: Promo, dtype: int64

In [44]:
train_int_data.groupby('purchase').count()

,Promo,V2,V3
purchase,,,
0,83494,83494,83494
1,1040,1040,1040


In [4]:
purchase_treat=train_data.loc[train_data['Promo'] == 1,'purchase'].sum()
customer_treat=train_data.loc[train_data['Promo'] == 1,'purchase'].count()
purchase_control= train_data.loc[train_data['Promo'] == 0,'purchase'].sum()
customer_control= train_data.loc[train_data['Promo'] == 0,'purchase'].count()
IRR=(purchase_treat/customer_treat-purchase_control/customer_control)
print(IRR)

0.00945454781977


In [58]:
train_data.loc[train_data['Promo'] == 1,'purchase'].sum()

721

In [56]:
train_data.loc[train_data['Promo'] == 1,'purchase'].count()

42364

In [59]:
train_data.loc[train_data['Promo'] == 0,'purchase'].sum()

319

In [60]:
train_data.loc[train_data['Promo'] == 0,'purchase'].count()

42170

In [5]:
NIR= (10*purchase_treat - 0.15*customer_treat)-10*purchase_control
print(NIR)

-2334.6


In [12]:
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier


X =  train_data[['V1','V2','V3','V4','V5','V6','V7']]
y = train_data[['purchase']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

In [7]:
from sklearn import svm, metrics

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred =clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.987697403442


In [19]:

from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm, metrics


clf = AdaBoostClassifier(learning_rate=0.01,n_estimators=200)
clf.fit(X_train, y_train)
y_pred =clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.987697403442


In [21]:
clf.score(X_test,y_test)


0.98769740344236112

In [26]:
def promotion_strategy(df):
    from sklearn.ensemble import AdaBoostClassifier
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    X =  df[['V1','V2','V3','V4','V5','V6','V7']]
    y = df[['purchase']]
    purchases=[]
    
    model = AdaBoostClassifier(learning_rate=0.01, n_estimators=200)
    model.fit(X, y)
    purchases = model.predict(df)
    print(purchases)
    #promotion = pd.Series(purchase).replace({0:'No', 1:'Yes'})
    
    
    
    return promotion

In [27]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy(train_data))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: could not convert string to float: 'No'